In [1]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
from torcheval.metrics import R2Score
import pandas as pd
import torch
from torch.utils.data import random_split
import shutil
import os
from dataset import PGLSDataset
from models import PGLSModel, SimpleTabularModel

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
for folder in ["data/train_images", "data/test_images"]:
    if "0" not in os.listdir(folder):
        print("Moving images to 0 folder")
        os.makedirs(f"{folder}/0")
        for filename in os.listdir(folder):
            if filename.lower().endswith(".jpeg"):
                source_path = os.path.join(folder, filename)
                target_path = os.path.join(f"{folder}/0", filename)

                shutil.move(source_path, target_path)

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor()            # Convert PIL image to tensor (H x W x C) in the range [0.0, 1.0]
])

batch_size = 4

In [5]:
train_images_path = 'data/train_images'
test_images_path = 'data/test_images'

train_csv_path = 'data/train.csv'
test_csv_path = 'data/test.csv'


tabular_data = pd.read_csv(train_csv_path)
targets = ["X4", "X11", "X18", "X26", "X50", "X3112"]

# Filter data
upper_values = {}
for target in targets:
    upper_values[target] = tabular_data[target+"_mean"].quantile(0.99)
    tabular_data = tabular_data[tabular_data[target+"_mean"] < upper_values[target]]
    tabular_data = tabular_data[tabular_data[target+"_mean"] > 0]

# Normalize the targets
original_means = tabular_data[[f"{target}_mean" for target in targets]].mean()
original_stds = tabular_data[[f"{target}_mean" for target in targets]].std()
tabular_data[[f"{target}_mean" for target in targets]] = (tabular_data[[f"{target}_mean" for target in targets]] - original_means) / original_stds

# Normalize the features
tabular_input_size = 0
for column in tabular_data.columns:
    if column in ["id"]+[target+"_mean" for target in targets]+[target+"_sd" for target in targets]:
        continue
    tabular_input_size += 1
    min_val = tabular_data[column].min()
    max_val = tabular_data[column].max()
    tabular_data[column] = (tabular_data[column] - min_val) / (max_val - min_val)


test_tabular_data = pd.read_csv(test_csv_path)
# Normalize the features
for column in test_tabular_data.columns:
    if column in ["id"]:
        continue
    min_val = test_tabular_data[column].min()
    max_val = test_tabular_data[column].max()
    test_tabular_data[column] = (test_tabular_data[column] - min_val) / (max_val - min_val)

train_images_dataset = ImageFolder(root=train_images_path, transform=transform)
test_images_dataset = ImageFolder(root=test_images_path, transform=transform)

train_image_csv_dataset = PGLSDataset(tabular_data=tabular_data, image_folder=train_images_dataset, transform_csv=None)
train, val = random_split(train_image_csv_dataset, [int(0.8*len(train_image_csv_dataset)), len(train_image_csv_dataset) - int(0.8*len(train_image_csv_dataset))])
test_image_csv_dataset = PGLSDataset(tabular_data=test_tabular_data, image_folder=test_images_dataset, transform_csv=None)


train_data_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_image_csv_dataset, batch_size=batch_size, shuffle=False)

In [6]:
def denormalize_targets(targets):
    return targets * original_stds.values + original_means.values

In [7]:
effnet = efficientnet_b0(weights=EfficientNet_B0_Weights)
tabular_model = SimpleTabularModel(input_data_len=tabular_input_size)
model = PGLSModel(effnet, tabular_model)
metric = R2Score()
torch.cuda.empty_cache()

/home/szaryvip/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(2):
    print(f"Epoch {epoch}")
    for data in train_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(image, features)
        loss = torch.nn.functional.mse_loss(outputs, targets)
        loss.backward()
        optimizer.step()
        print(loss.item())
        print(metric.update(denormalize_targets(outputs), denormalize_targets(targets)).compute())


Epoch 0


/home/szaryvip/.local/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


1.0196490287780762
tensor(-0.9337, device='cuda:0')
0.789876401424408
tensor(-1.2439, device='cuda:0')
0.9147118330001831
tensor(-1.8074, device='cuda:0')
0.698845386505127
tensor(-2.1408, device='cuda:0')
2.4834511280059814
tensor(-2.4964, device='cuda:0')
0.8903217315673828
tensor(-2.7147, device='cuda:0')
1.226850986480713
tensor(-2.1878, device='cuda:0')
1.3428866863250732
tensor(-2.4602, device='cuda:0')
1.816765546798706
tensor(-2.1124, device='cuda:0')
1.7165488004684448
tensor(-1.9789, device='cuda:0')
1.105561375617981
tensor(-1.8394, device='cuda:0')
2.7942023277282715
tensor(-0.9143, device='cuda:0')
1.034125804901123
tensor(-0.9956, device='cuda:0')
1.6941889524459839
tensor(-0.7878, device='cuda:0')
2.5320074558258057
tensor(-0.8059, device='cuda:0')
2.008476734161377
tensor(-0.7988, device='cuda:0')
2.537299633026123
tensor(-0.7797, device='cuda:0')
1.88002347946167
tensor(-0.8041, device='cuda:0')
2.558772087097168
tensor(-0.8161, device='cuda:0')
0.7459964752197266
tens

KeyboardInterrupt: 

In [36]:
accumulated_loss = 0
iterations = 0
accumulated_r2 = 0
model.to(device)
model.eval()
for data in val_data_loader:
    image, features, targets = data
    image = image.to(device)
    features = features.to(device)
    targets = targets.to(device)
    outputs = model(image, features)
    loss = torch.nn.functional.mse_loss(outputs, targets)
    accumulated_loss += loss.item()
    iterations += 1
    outputs = outputs.cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    outputs_denorm = torch.from_numpy(denormalize_targets(outputs))
    targets_denorm = torch.from_numpy(denormalize_targets(targets))
    metric.update(outputs_denorm, targets_denorm)
    accumulated_r2 += metric.compute().item()
print("Average loss on validation set: ", accumulated_loss/iterations)
print("Average R2 on validation set: ", accumulated_r2/iterations)

Average loss on validation set:  1.7237616377462581
Average R2 on validation set:  tensor(-1.1333, dtype=torch.float64)


In [8]:
predictions = {}
model.eval()
model.to(device)
for data in test_data_loader:
    image, features, targets = data
    image = image.to(device)
    features = features.to(device)
    targets = targets.to(device)
    outputs = model(image, features)
    outputs = outputs.cpu().detach().numpy()
    outputs_denorm = denormalize_targets(outputs)
    predictions.append(outputs_denorm)

In [13]:
predictions = [item for sublist in predictions for item in sublist]

In [14]:
predictions

[array([  0.61058684,  14.96380795,   6.83026935,  20.24349493,
          1.12333255, 327.37596666]),
 array([ 5.99621037e-01,  2.02214342e+01, -5.55097035e-02,  8.35147105e+01,
         1.03436920e+00,  2.18780669e+03]),
 array([   0.52866967,   16.63509594,   -2.92241581,    6.98299332,
           0.4405925 , -295.53439051]),
 array([ 6.30189980e-01,  1.38502444e+01, -1.05987100e+00, -2.49329927e+00,
         1.68390033e+00, -1.28081142e+03]),
 array([ 5.61827109e-01,  1.89652758e+01, -6.41808785e-01,  4.23484121e+01,
         1.74719844e+00,  1.87453515e+03]),
 array([ 3.25144603e-01,  1.17547952e+01,  6.49758621e+00,  9.07215614e+01,
         8.10393909e-01, -1.01466091e+03]),
 array([ 5.90099782e-01,  1.63377484e+01,  1.18484297e+00, -1.58926933e+01,
         1.09476609e+00, -1.48294104e+03]),
 array([ 5.01498615e-01,  5.11384323e+00,  6.49339492e-01,  3.42413353e+00,
         1.90968197e+00, -1.47911922e+03]),
 array([ 0.57533319, 21.08796138, -2.01432915, 68.20504329,  1.4860191

In [10]:
def save_predictions(predictions, test_csv_dataframe, filename):
    with open(filename, "w") as f:
        f.write("id,X4,X11,X18,X26,X50,X3112\n")
        for pred, id in zip(predictions, test_csv_dataframe["id"]):
            f.write(f"{id},{','.join([str(p) for p in pred])}\n")

In [15]:
save_predictions(predictions, test_tabular_data, "predictions.csv")